In [2]:
import pandas as pd
import numpy as np

import os

os.chdir("C:\\EclipseBytes\\CodeBaseDebo\\sql_excel")


In [12]:
zipcensus = pd.read_csv('zipcensus.txt',sep='\t')
#print(zipcensus.head(5))
zipcensus.shape

(32038, 113)

In [4]:
# to reset index
df1 = zipcensus[['zipcode','state',]].groupby('state').count().reset_index()


# without resetting the index
df = zipcensus[['zipcode','state',]].groupby('state').count()
df.sort_values(by=['zipcode'],ascending=[False]).head(5)

,zipcode
state,
TX,1865
PA,1722
CA,1677
NY,1619
IL,1347


In [8]:
#zipcensus.shape
#len(np.unique(zipcensus['zipcode'].str[:3]))

#df1.head(5)
df1['ini_zip'] = df1['zipcode'].apply(str).str[:3]
#df1.head(5)

df2 = df1[['state','ini_zip']].groupby(['state','ini_zip']).count()
df2 = df2.reset_index()
df2.sort_values(by=['state']).head(5)
#df2.sort_values(by=['ini_zip'],ascending=[False]).head(5)

#df1['zipcode'].apply(str).str[:3]

,state,ini_zip
0,AK,241
1,AL,624
2,AR,591
3,AZ,362
4,CA,167


In [13]:
zipcensus.columns

Index(['zipcode', 'state', 'population', 'housingunits', 'landareameters',
       'waterareameters', 'landareamiles', 'waterareamiles', 'latitude',
       'longitude',
       ...
       'hhuofuelnone', 'hhuoplumbingcomplete', 'hhuoplumbinglacking',
       'hhumediancashrent', 'hhunmarried', 'hhunmarriedmm', 'hhunmarriedmf',
       'hhunmarriedff', 'hhunmarriedfm', 'hhother'],
      dtype='object', length=113)

In [38]:
def cal10000(d):
    return sum(np.where(d>10000,1,0))
def cal1000(d):
    return sum(np.where(d>1000,1,0))

def pop10000(d):
    return sum(np.where(d >10000,d,0))
def pop1000(d):
    return sum(np.where(d >1000,d,0))



In [42]:
import numpy as np
zipcensus[["state","population"]].groupby(["state"]).agg({"population":[cal10000,cal1000,pop10000,pop1000]}).reset_index().head(5)

state population                            
          cal10000 cal1000  pop10000   pop1000
0    AK         20      60    418751    565203
1    AL        157     513   2911817   4392989
2    AR         79     344   1678564   2568178
3    AZ        152     295   4540120   5102380
4    CA        928    1344  31998203  33734836

In [138]:
import random
import pandas as pd

x=pd.DataFrame(np.random.randint(0,2,size=(133,1)),columns=list('A'))
x.head()

def cal10(d):
    c = list(d)
    e={"cnt1":c.count(1),"cnt0":c.count(0),"count":len(c),"pct1":c.count(1)/len(c),"pct0":c.count(0)/len(c)}
    #d ={"cnt1":sum(np.where(d==1,1,0)),"cnt0":sum(np.where(d==0,1,0)),"pct1":sum(np.where(d==1,1,0))/len(d)}
    return e

data = cal10(x['A'])
pd.DataFrame([data])


,cnt0,cnt1,count,pct0,pct1
0,73,60,133,0.548872,0.451128


In [207]:
sub_zip = zipcensus[["state","population"]].copy()


sub_zip["is_pop_10000"] = np.where(sub_zip["population"].values >= 10000,1,0)
sub_zip["is_pop_1000"] = np.where(sub_zip["population"].values >= 1000,1,0)

sub_zip["pop_10000"] = sub_zip["population"]*sub_zip["is_pop_10000"]
sub_zip["pop_1000"] = sub_zip["population"]*sub_zip["is_pop_1000"]


df = sub_zip.groupby(["state"]).agg({"is_pop_10000":sum,"is_pop_1000":sum,"pop_10000":sum,"pop_1000":sum})
df.head(5)



,is_pop_10000,pop_10000,pop_1000,is_pop_1000
state,,,,
AK,20,418751,565203,60
AL,157,2911817,4392989,513
AR,79,1678564,2568178,344
AZ,152,4540120,5102380,295
CA,928,31998203,33735836,1345


In [145]:
df1.head(3)

,state,zipcode,ini_zip
0,AK,241,241
1,AL,624,624
2,AR,591,591


In [213]:
zippop = zipcensus[['state','population','landareamiles']]
zippop = zippop.copy()

#zippop = zippop.set_index(['state','population'])

zippop['popdensity'] = pd.Series(zippop['population']/zippop['landareamiles'])

#del zippop['avg_popden']

temp = zippop[['state','popdensity']].groupby(['state']).agg([np.mean]).reset_index()

zippop.loc[:,'avg_popden'] = zippop[['state','popdensity']].groupby(['state']).transform(np.mean)

print(temp.head(5))
print('\n')
print(temp[temp['state']=='AK'])
print('\n')

print(zippop.head(5))
print('\n')
print(zippop[zippop['state']=='AK'].head(5))

#def chk_b

print(zippop[['state','popdensity','avg_popden']].groupby(['state']).agg({'popdensity':[np.size]}).head(5))

#final = pd.merge(zippop,temp,on=['state'],how='inner')
#print(final.head(5))
#print('\n')

  state   popdensity
                mean
0    AK   243.659722
1    AL   313.041088
2    AR   145.443226
3    AZ  1098.836658
4    CA  3179.195724


  state  popdensity
               mean
0    AK  243.659722


  state  population  landareamiles   popdensity  avg_popden
0    PR       19143      66.691965   287.036077         NaN
1    PR       42042      30.941214  1358.770215         NaN
2    PR       55530      30.383543  1827.634124         NaN
3    PR        3923       3.005935  1305.084774         NaN
4    PR        6449      36.629531   176.060130         NaN


      state  population  landareamiles   popdensity  avg_popden
31797    AK       16211       6.321246  2564.526044         NaN
31798    AK       18626      23.156015   804.369837         NaN
31799    AK       13847       3.334455  4152.702616         NaN
31800    AK       37284      73.931765   504.302853         NaN
31801    AK        5322      63.236742    84.159933         NaN
      popdensity
            size
state    

In [1]:
zipcensus['zipcode'] = zipcensus['zipcode'].astype(str)
zipcensus['zipcode'].map(len)[:3]

NameError: name 'zipcensus' is not defined